# 环境

In [1]:
import os 
import copy 
import torch
import argparse
import numpy as np
import torch.nn as nn
from timm.models import create_model
from datasets import build_dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, Dataset,TensorDataset
from torchvision.datasets.folder import ImageFolder

import models
from pruning_utils import *
from layers import Conv2d, Linear
from sp_vision_transformer import Attention, Mlp

lxytimm-init.py-timm.__version__ 0.6.13


In [ ]:
# from timm.models import create_model, safe_model_name, resume_checkpoint, load_checkpoint, model_parameters
# from timm.data import create_dataset, create_loader, resolve_data_config

# 参数

In [2]:
#手动定义参数类，专门存储训练的参数，
class cfg(object):
    pass
args = cfg()#实例化
# args.epochs, args.learning_rate, args.patience = [10, 0.001, 4]
# args.hidden_size, args.input_size= [40, 30]

# args.model = 'vit_small_patch16_224'
# args.dataset = 'torch/CIFAR10'
# args.data_dir = '../datasets'
# args.batch_size = 32
# args.pin_mem = 10


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#strip/gmp/amp_50_small.sh
args.sparsity=0.4
args.atten_density=0.5
args.mlp_density=0.5
args.heads=12
args.pretrained='https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth'
args.save_file='deit_base_patch16_224_omp40_mag.pt'
args.data=None
args.batch_size=128
args.type='mag'
args.model='deit_base_patch16_224'
args.input_size=224
args.drop=0.0
args.drop_path=0.1

In [3]:
print(args)

# 函数

In [4]:
def check_atten(binary_tensor, dim=64):
    heads_num = binary_tensor.shape[0] // dim
    output_list = []
    for i in range(heads_num):
        activated = binary_tensor[i*dim:i*dim+dim].mean().byte().float()
        output_list.append(activated.item())
    return output_list

def prune_loop(model, loss, pruner, dataloader, device, sparsity, scope, epochs, train_mode=False):

    # Set model to train or eval mode
    model.train()
    if not train_mode:
        model.eval()

    # Prune model
    for epoch in range(epochs):
        pruner.score(model, loss, dataloader, device)
        sparse = sparsity**((epoch + 1) / epochs) # represent density actually.
        pruner.mask(sparse, scope)

def prune_conv_linear(model):

    for name, module in reversed(model._modules.items()):

        if len(list(module.children())) > 0:
            model._modules[name] = prune_conv_linear(model=module)

        if isinstance(module, nn.Linear):
            bias=True
            if module.bias == None:
                bias=False
            layer_new = Linear(module.in_features, module.out_features, bias)
            model._modules[name] = layer_new

        if isinstance(module, nn.Conv2d):
            layer_new = Conv2d(module.in_channels, module.out_channels, module.kernel_size, module.stride)
            model._modules[name] = layer_new

    return model

# main

In [5]:
if args.type == 'mag':

    print('#########################################')
    print('########## Magnitude ####################')
    print('#########################################')

    number_examples = 128
    data = torch.ones(number_examples, 3, args.input_size, args.input_size)
    target = torch.ones(number_examples)
    data_set = torch.utils.data.TensorDataset(data, target)
    loader = torch.utils.data.DataLoader(data_set, 
        batch_size=1, shuffle=False, 
        num_workers=2, pin_memory=True)


#########################################
########## Magnitude ####################
#########################################


In [4]:

if args.type == 'mag':
    print(f"Creating model: {args.model}")
    model = create_model(
        args.model,
        pretrained=False,
        num_classes=1000,
        drop_rate=args.drop,
        drop_path_rate=args.drop_path,
        drop_block_rate=None,
    )
    


Creating model: deit_base_patch16_224


In [7]:
# if 1:
#     from vision_transformer import Attention
#     #from sp_vision_transformer import Attention
#     for name, m in model.named_modules():
#         if isinstance(m, Attention):
#             print(name)

blocks.0.attn
blocks.1.attn
blocks.2.attn
blocks.3.attn
blocks.4.attn
blocks.5.attn
blocks.6.attn
blocks.7.attn
blocks.8.attn
blocks.9.attn
blocks.10.attn
blocks.11.attn


In [7]:
model._modules

OrderedDict([('patch_embed',
              PatchEmbed(
                (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
              )),
             ('pos_drop', Dropout(p=0.0, inplace=False)),
             ('gumbel', Linear(in_features=768, out_features=1, bias=True)),
             ('blocks',
              Sequential(
                (0): Block(
                  (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
                  (attn): Attention(
                    (qkv): Linear(in_features=768, out_features=2304, bias=True)
                    (attn_drop): Dropout(p=0.0, inplace=False)
                    (proj): Linear(in_features=768, out_features=768, bias=True)
                    (proj_drop): Dropout(p=0.0, inplace=False)
                  )
                  (drop_path): Identity()
                  (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
                  (mlp): Mlp(
                    (fc1): Linear(in_features=768, out_fe

In [8]:
for name, module in model._modules.items():
    print(name)
'''
patch_embed
pos_drop
gumbel
blocks
norm
pre_logits
head

'''



patch_embed
pos_drop
gumbel
blocks
norm
pre_logits
head


'\npatch_embed\npos_drop\ngumbel\nblocks\nnorm\npre_logits\nhead\n\n'

## load pretrained weight

In [9]:
if args.type == 'mag':
    print('loading pretrained weight')
    checkpoint = torch.hub.load_state_dict_from_url(
        args.pretrained, map_location='cpu', check_hash=True)
    model.load_state_dict(checkpoint['model'],strict=False)#改了一下，不然提示Missing key(s) in state_dict: "gumbel.weight", "gumbel.bias". 

    save_state_dict = copy.deepcopy(model.state_dict())


loading pretrained weight


## 替换模型的conv和linear？？
看layers.py ，新建的层里面有self.register_buffer('weight_mask', torch.ones(self.weight.shape))

In [10]:

if args.type == 'mag':    
    prune_conv_linear(model)#moedl._modules[name] = layer_new  新的conv fc


## 把之前deepcopy的参数重新加载回去

In [11]:

if args.type == 'mag':
    for key in save_state_dict.keys():
        if not key in model.state_dict().keys():
            print('can not load key = {}'.format(key))

    model.load_state_dict(save_state_dict, strict=False)
    model.cuda()


## pruner包装

masked_parameters()
Returns an iterator over models prunable parameters, yielding both the mask and parameter tensors.

Mag基于Pruner（见pruning_utils.py）  ,主要重写的score

类似的
https://github.com/facebookresearch/open_lth/blob/main/pruning/test/test_pruned_model.py

https://github.com/facebookresearch/open_lth

王欢-https://github.com/MingSun-Tse

https://github.com/MingSun-Tse/Regularization-Pruning

https://github.com/MingSun-Tse/Why-the-State-of-Pruning-so-Confusing


196的图剪枝，也实现了一些基础的pruner

In [12]:

if args.type == 'mag':
    pruner = Mag(masked_parameters(model))


In [25]:
for mask, param  in pruner.masked_parameters:
    #print(mask.size(), param.size())
    print(param.size())

    #PatchEmbed((proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))#torch.Size([768, 3, 16, 16])
    #torch.Size([1, 768]) 'gumbel'
    # torch.Size([2304, 768])qkv
    # torch.Size([768, 768])proj
    #torch.Size([3072, 768]) fc1
    #torch.Size([768, 3072]) fc2

torch.Size([768, 3, 16, 16])
torch.Size([1, 768])
torch.Size([2304, 768])
torch.Size([768, 768])
torch.Size([3072, 768])
torch.Size([768, 3072])
torch.Size([2304, 768])
torch.Size([768, 768])
torch.Size([3072, 768])
torch.Size([768, 3072])
torch.Size([2304, 768])
torch.Size([768, 768])
torch.Size([3072, 768])
torch.Size([768, 3072])
torch.Size([2304, 768])
torch.Size([768, 768])
torch.Size([3072, 768])
torch.Size([768, 3072])
torch.Size([2304, 768])
torch.Size([768, 768])
torch.Size([3072, 768])
torch.Size([768, 3072])
torch.Size([2304, 768])
torch.Size([768, 768])
torch.Size([3072, 768])
torch.Size([768, 3072])
torch.Size([2304, 768])
torch.Size([768, 768])
torch.Size([3072, 768])
torch.Size([768, 3072])
torch.Size([2304, 768])
torch.Size([768, 768])
torch.Size([3072, 768])
torch.Size([768, 3072])
torch.Size([2304, 768])
torch.Size([768, 768])
torch.Size([3072, 768])
torch.Size([768, 3072])
torch.Size([2304, 768])
torch.Size([768, 768])
torch.Size([3072, 768])
torch.Size([768, 3072])


In [27]:
pruner.scores#{}

{140413212058144: tensor([[[[4.3330e-03, 1.0408e-02, 1.2760e-02,  ..., 4.2243e-03,
            1.6184e-02, 1.7052e-02],
           [5.6061e-03, 1.4428e-02, 1.3019e-02,  ..., 1.1680e-02,
            1.2779e-02, 2.2419e-02],
           [3.2034e-03, 1.3556e-02, 1.1541e-03,  ..., 9.7929e-03,
            2.3665e-03, 2.1954e-02],
           ...,
           [8.9069e-03, 2.6199e-03, 9.1569e-03,  ..., 1.6778e-02,
            3.8766e-03, 1.2634e-02],
           [6.8154e-03, 1.3984e-03, 6.2971e-03,  ..., 8.7928e-03,
            1.4149e-02, 1.3649e-02],
           [1.9803e-02, 1.5871e-02, 1.9681e-03,  ..., 3.8663e-03,
            1.1515e-02, 1.7161e-02]],
 
          [[1.3695e-02, 1.2757e-02, 9.7532e-03,  ..., 1.5428e-03,
            3.3857e-03, 9.3559e-04],
           [4.0775e-03, 8.4886e-03, 4.7667e-03,  ..., 1.1830e-02,
            8.1848e-03, 6.3493e-03],
           [1.5393e-02, 9.5510e-03, 6.3038e-03,  ..., 3.5832e-03,
            7.6596e-03, 4.5198e-03],
           ...,
           [6.6670e-0

## loop 看源码 拆开长这个样子

In [ ]:

# if args.type == 'mag':
#     prune_loop(model, None, pruner, loader, torch.device('cuda:0'), args.sparsity, scope='global', epochs=1, train_mode=True)
#     print('Density = {}'.format(args.sparsity))


In [14]:
epoch = 0
epochs = 1
scope='global'
model.train()


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (gumbel): Linear(in_features=768, out_features=1, bias=True)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNo

In [15]:

pruner.score(model, None, loader, torch.device('cuda:0'))



In [26]:
for k in pruner.scores.keys():
    print(k)
    print(pruner.scores[k].size())
    if k == 140413138820928:
        print(pruner.scores[k])



140413212058144
torch.Size([768, 3, 16, 16])
140413138820928
torch.Size([1, 768])
tensor([[1.7929e-02, 1.8647e-02, 5.0791e-03, 8.5352e-03, 3.1633e-02, 1.0669e-02,
         2.6471e-02, 8.0351e-03, 2.3983e-03, 2.4840e-03, 1.1692e-02, 3.3947e-02,
         2.5252e-02, 2.1454e-02, 3.4516e-02, 1.7447e-02, 3.9693e-03, 1.6686e-02,
         9.6017e-04, 1.8976e-02, 8.3181e-03, 5.5111e-03, 1.3257e-02, 3.5588e-03,
         4.7560e-02, 1.8541e-02, 1.9805e-02, 1.3845e-02, 2.8073e-02, 1.0827e-02,
         1.9446e-04, 1.9278e-02, 1.6585e-02, 2.0620e-02, 2.4384e-02, 7.6181e-03,
         2.0938e-02, 4.5046e-03, 3.0626e-02, 1.9054e-02, 5.2162e-02, 3.4482e-02,
         2.2695e-03, 2.8176e-02, 1.3094e-02, 1.1418e-02, 3.4662e-02, 3.6163e-02,
         1.2381e-02, 1.3419e-02, 4.0565e-02, 1.7143e-02, 5.2222e-03, 1.5338e-02,
         2.7271e-02, 8.8123e-03, 3.9367e-02, 1.1375e-02, 5.5016e-03, 2.3066e-02,
         1.4849e-02, 2.3158e-02, 4.6975e-03, 2.1393e-02, 3.8825e-02, 8.7000e-03,
         7.9070e-04, 1.2772

In [24]:
sparse = args.sparsity**((epoch + 1) / epochs) # represent density actually. --0.4


In [25]:

pruner.mask(sparse, scope)

print('Density = {}'.format(args.sparsity))

Density = 0.4


## 后处理

In [29]:

if args.type == 'mag':
    current_mask = extract_mask(model.state_dict())
    check_sparsity_dict(current_mask)# 打印一下实际剩余参数占原本参数的rate 有没有到之前的 sparsity


* remain weight =  39.99999976823215 %


In [30]:
current_mask#'patch_embed.proj.weight_mask': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.], 单个元素

{'patch_embed.proj.weight_mask': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 1.,  ..., 1., 1., 0.],
           [0., 0., 1.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
     

## 保存

In [ ]:

if args.type == 'mag':
    torch.save(current_mask, args.save_file)